In [3]:
import os
import ast
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np

SCORES_DIR = Path("scores")

def parse_model_name(filename: str) -> str:
    """
    Derive model name from filenames like:
      litellm_MODEL_xxx.csv  or  vllm_MODEL_xxx.csv
    Drops the 'litellm_'/'vllm_' prefix and the last underscore chunk (often a run id/timestamp).
    """
    base = Path(filename).stem
    for prefix in ("litellm_", "vllm_"):
        if base.startswith(prefix):
            base = base[len(prefix):]
    parts = base.split("_")
    return "_".join(parts[:-1]) if len(parts) > 1 else base

def get_score(s) -> float:
    """
    Safely convert various 'score' representations to float.
    Accepts numbers, numeric strings, or dict-like strings with 'value' (e.g., "{'value': 0.83}").
    """
    if pd.isna(s):
        return 0.0
    if isinstance(s, (int, float, np.number)):
        return float(s)
    s = str(s).strip()
    # Try dict-like with 'value'
    try:
        d = ast.literal_eval(s)
        if isinstance(d, dict) and "value" in d:
            return float(d["value"])
    except Exception:
        pass
    # Fallback: try to parse as plain float
    try:
        return float(s)
    except Exception:
        return 0.0

# Aggregators
overall = defaultdict(lambda: {"sum": 0.0, "cnt": 0})
by_cat = defaultdict(lambda: defaultdict(lambda: {"sum": 0.0, "cnt": 0}))

for fp in tqdm(os.listdir(SCORES_DIR)):
    if not fp.endswith(".csv"):
        continue

    df = pd.read_csv(SCORES_DIR / fp)
    if "score" not in df.columns or "category" not in df.columns:
        # Skip files that don't have the needed columns
        continue

    df["score_num"] = df["score"].apply(get_score)
    model = parse_model_name(fp)

    # Overall aggregation
    overall[model]["sum"] += df["score_num"].sum()
    overall[model]["cnt"] += df["score_num"].shape[0]

    # Per-category aggregation
    for cat, grp in df.groupby("category", dropna=False):
        by_cat[model][cat]["sum"] += grp["score_num"].sum()
        by_cat[model][cat]["cnt"] += grp["score_num"].shape[0]

# Finalize means
model_overall = {
    m: (v["sum"] / v["cnt"]) if v["cnt"] else 0.0
    for m, v in overall.items()
}
model_by_category = {
    m: {c: (vc["sum"] / vc["cnt"]) if vc["cnt"] else 0.0 for c, vc in cats.items()}
    for m, cats in by_cat.items()
}

# (Optional) Pretty DataFrames
overall_df = (
    pd.DataFrame([{"model": m, "score": s} for m, s in model_overall.items()])
      .sort_values("score", ascending=False)
      .reset_index(drop=True)
)

bycat_rows = []
for m, cats in model_by_category.items():
    for c, s in cats.items():
        bycat_rows.append({"model": m, "category": c, "score": s})
bycat_df = (
    pd.DataFrame(bycat_rows)
      .sort_values(["model", "category"])
      .reset_index(drop=True)
)

print("Overall (mean score per model):\n", overall_df)
print("\nBy Category (mean score per model & category):\n", bycat_df)

# Wide table: model (rows) × category (columns)
wide_df = (
    bycat_df.pivot_table(index="model", columns="category", values="score", aggfunc="mean")
            .sort_index()
            .sort_index(axis=1)
)

# (Optional) add overall mean as the leftmost column
overall_s = overall_df.set_index("model")["score"]
wide_df = overall_s.to_frame("Overall").join(wide_df)

# (Optional) tidy up
wide_df = wide_df.round(4).fillna(0.0)

wide_df.to_excel("scores_summary_by_category_gemini25flash.xlsx")



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [04:02<00:00,  1.99s/it]

Overall (mean score per model):
                                                 model     score
0                               gemini_gemini-2.5-pro  0.485430
1                                               gpt-5  0.480139
2                                          gpt-5-mini  0.452082
3                     openrouter_perplexity_sonar-pro  0.442577
4                             gemini_gemini-2.5-flash  0.410265
5                              openrouter_x-ai_grok-4  0.360620
6                        gemini_gemini-2.5-flash-lite  0.302948
7                            Skywork_Skywork-R1V3-38B  0.277631
8             openrouter_mistralai_mistral-medium-3.1  0.248612
9                               google_gemma-3-27b-it  0.225179
10                                         gpt-5-nano  0.212170
11            openrouter_qwen_qwen2.5-vl-72b-instruct  0.205789
12    openrouter_qwen_qwen2.5-vl-72b-instruct_oneline  0.202297
13            openrouter_mistralai_pixtral-large-2411  0.200961
14     

In [5]:
import pandas as pd

df=pd.read_csv('scores_gemini/vllm_Qwen_Qwen2.5-VL-3B-Instruct_1.csv')

In [9]:
import os
import ast
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm
import pandas as pd
import numpy as np

SCORES_DIR = Path("scores")

def parse_model_name(filename: str) -> str:
    """
    Derive model name from filenames like:
      litellm_MODEL_xxx.csv  or  vllm_MODEL_xxx.csv
    Drops the 'litellm_'/'vllm_' prefix and the last underscore chunk (often a run id/timestamp).
    """
    base = Path(filename).stem
    for prefix in ("litellm_", "vllm_"):
        if base.startswith(prefix):
            base = base[len(prefix):]
    parts = base.split("_")
    return "_".join(parts[:-1]) if len(parts) > 1 else base

def get_score(s) -> float:
    """
    Safely convert various 'score' representations to float.
    Accepts numbers, numeric strings, or dict-like strings with 'value' (e.g., "{'value': 0.83}").
    """
    if pd.isna(s):
        return 0.0
    if isinstance(s, (int, float, np.number)):
        return float(s)
    s = str(s).strip()
    # Try dict-like with 'value'
    try:
        d = ast.literal_eval(s)
        if isinstance(d, dict) and "value" in d:
            return float(d["value"])
    except Exception:
        pass
    # Fallback: try to parse as plain float
    try:
        return float(s)
    except Exception:
        return 0.0

class RunningStats:
    """Welford's online algorithm for mean/std (sample std, ddof=1)."""
    __slots__ = ("n", "mean", "M2")
    def __init__(self):
        self.n = 0
        self.mean = 0.0
        self.M2 = 0.0
    def update(self, x: float):
        self.n += 1
        delta = x - self.mean
        self.mean += delta / self.n
        delta2 = x - self.mean
        self.M2 += delta * delta2
    @property
    def count(self) -> int:
        return self.n
    @property
    def var(self) -> float:
        if self.n > 1:
            return self.M2 / (self.n - 1)
        return np.nan
    @property
    def std(self) -> float:
        v = self.var
        return float(np.sqrt(v)) if not np.isnan(v) else np.nan

# Aggregators: model-level and model→category-level
overall = defaultdict(RunningStats)
by_cat = defaultdict(lambda: defaultdict(RunningStats))

for fp in tqdm(os.listdir(SCORES_DIR)):
    if not fp.endswith(".csv"):
        continue

    df = pd.read_csv(SCORES_DIR / fp)
    if "score" not in df.columns or "category" not in df.columns:
        # Skip files that don't have the needed columns
        continue

    df["score_num"] = df["score"].apply(get_score)
    model = parse_model_name(fp)

    # Update overall stats
    for x in df["score_num"].values:
        overall[model].update(float(x))

    # Update per-category stats
    for cat, grp in df.groupby("category", dropna=False):
        rs = by_cat[model][cat]
        for x in grp["score_num"].values:
            rs.update(float(x))

# Build tidy DataFrames
overall_rows = []
for m, rs in overall.items():
    overall_rows.append({
        "model": m,
        "mean": rs.mean,
        "std": rs.std,
        "n": rs.count,
    })
overall_df = (
    pd.DataFrame(overall_rows)
      .sort_values("mean", ascending=False)
      .reset_index(drop=True)
)

bycat_rows = []
for m, cats in by_cat.items():
    for c, rs in cats.items():
        bycat_rows.append({
            "model": m,
            "category": c,
            "mean": rs.mean,
            "std": rs.std,
            "n": rs.count,
        })
bycat_df = (
    pd.DataFrame(bycat_rows)
      .sort_values(["model", "category"])
      .reset_index(drop=True)
)

print("Overall (mean/std per model):\n", overall_df)
print("\nBy Category (mean/std per model & category):\n", bycat_df)

# Wide table with MultiIndex columns: (Category, Metric) where Metric ∈ {mean, std}
pivot = bycat_df.pivot_table(
    index="model",
    columns="category",
    values=["mean", "std"],
    aggfunc="first"
)

# Make columns as (Category, Metric) instead of (Metric, Category)
pivot = pivot.swaplevel(0, 1, axis=1).sort_index(axis=1, level=0)

# Add Overall group on the left
overall_pair = overall_df.set_index("model")[["mean", "std"]]
overall_pair.columns = pd.MultiIndex.from_product([["Overall"], overall_pair.columns])

wide_df = pd.concat([overall_pair, pivot], axis=1)

# Tidy up
wide_df = wide_df.round(4)

# Write to Excel
out_path = "scores_summary_by_category_gemini25flash.xlsx"
with pd.ExcelWriter(out_path, engine="xlsxwriter") as writer:
    overall_df.round(4).to_excel(writer, sheet_name="overall_mean_std", index=False)
    bycat_df.round(4).to_excel(writer, sheet_name="by_category_mean_std", index=False)
    wide_df.to_excel(writer, sheet_name="wide_mean_std")

print(f"\nSaved summary to: {out_path}")


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 122/122 [04:00<00:00,  1.97s/it]

Overall (mean/std per model):
                                                 model      mean       std  \
0                               gemini_gemini-2.5-pro  0.485430  0.361108   
1                                               gpt-5  0.480139  0.376761   
2                                          gpt-5-mini  0.452082  0.351147   
3                     openrouter_perplexity_sonar-pro  0.442577  0.339074   
4                             gemini_gemini-2.5-flash  0.410265  0.352886   
5                              openrouter_x-ai_grok-4  0.360620  0.350348   
6                        gemini_gemini-2.5-flash-lite  0.302948  0.306091   
7                            Skywork_Skywork-R1V3-38B  0.277631  0.287605   
8             openrouter_mistralai_mistral-medium-3.1  0.248612  0.283986   
9                               google_gemma-3-27b-it  0.225179  0.271094   
10                                         gpt-5-nano  0.212170  0.299328   
11            openrouter_qwen_qwen2.5-vl-72b-

In [8]:
pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
